![](../img/330-banner.png)

<br><br><br>
<img src="https://media.licdn.com/dms/image/C5622AQG0zUZ7YBpKMA/feedshare-shrink_800/0/1677468692008?e=1680739200&v=beta&t=s3RUU4qy0ky0D5b4y02bL9J2g2eeD4fQ1USrClhXB5I">
<br><br><br>

# Lecture 11: Ensembles
--------------

UBC 2022-23 W2

Instructor: Amir Abdi
 - Office Hours: Mondays 5-6 (or 5-7 if student turn-out was high)
 

<br><br><br><br><br><br>

## Legends

    
| <img src="https://upload.wikimedia.org/wikipedia/commons/f/f8/This_is_the_photo_of_Arthur_Samuel.jpg" width="100"> | <img src="http://www.cs.cmu.edu/~tom/TomHead2-6-22-22.jpg" width="100">  | <img src="https://upload.wikimedia.org/wikipedia/commons/4/49/John_McCarthy_Stanford.jpg" width="100"> | <img src="https://datascience.columbia.edu/wp-content/uploads/2020/08/Vapnik_web.png" width="100"> | <img src="https://upload.wikimedia.org/wikipedia/commons/a/a1/Alan_Turing_Aged_16.jpg" width="100"> | <img src="https://upload.wikimedia.org/wikipedia/commons/1/1e/Yoshua_Bengio_2019_cropped.jpg" width="100"> |
| :-----------: | :-----------: | :-----------: | :-----------: | :-----------: | :-----------: | 
| Arthur Samuel       | Tom Mitchell       |John McCarthy|  Vladimir N. Vapnik | Alan Turing | Yoshua Bengio |
| (1901-1990)    | 1951 - Now       |  1927 – 2011 | 1936 - Now | 1912 – 1954 | 1964-Now |
| First computer learning program | 1997 ML Texbook, CMU Prof | Co-coined term AI, Lisp,<br> Time-sharing, Garbage collection | SVM | Turing Test, Turning Machine | Turing Award<br> Father of Deep Learning


## Imports, announcements, LOs 

### Imports

In [ ]:
import os

%matplotlib inline
import string
import sys
from collections import deque

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../code/.")

from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier

### Announcements

- How was the midterm
- HW5 due hw5 March 1, 11:59pm

### Lecture learning objectives

From this lecture, you will be able to 

- Use `scikit-learn`'s `RandomForestClassifier` and explain its main hyperparameters. 
- Explain randomness in random forest algorithm. 
- Use other tree-based models such as as `XGBoost` and `LGBM`.  
- Employ ensemble classifier approaches, in particular model averaging and stacking.
- Explain voting and stacking and the differences between them.
- Use `scikit-learn` implementations of these ensemble methods.

## Motivation


- **Ensembles** are models that combine multiple machine learning models to create more powerful models.
- There are different strategies in Ensembling models:
  - Bagging
  - Boosting
  - Stackingz

<br><br><br><br><br><br><br><br>

### The Netflix prize

![](../img/netflix.png)

[Source](https://netflixtechblog.com/netflix-recommendations-beyond-the-5-stars-part-1-55838468f429)

- Most of the winning solutions for Kaggle competitions involve some kind of ensembling. For example: 


<img src="../img/fraud_detection_kaggle.png" width="600" height="600">


<br><br><br><br><br><br><br>
Key idea: **Group (ensemble) of Models** can make better decisions than **individual models**, especially when models are diverse enough. 


<br><br><br><br><br><br><br>

What's the cost? (why not always use ensembles?)

Answer: ?????

<br><br><br><br><br><br><br>


### Tree-based ensemble models 

- We'll briefly talk about two such models: 
    - Random forests
    - Gradient boosted trees
- We'll also talk about averaging (???) and stacking. 

### Tree-based models 

- Decision trees models are 
    - Interpretable 
    - They can capture **non-linear relationships**
    - They don't require scaling of the data
    - Can work with categorical features. 

<img src="https://cdck-file-uploads-global.s3.dualstack.us-west-2.amazonaws.com/business6/uploads/analyticsvidhya/original/2X/7/72d79a7b7841d04a077c5ac7a01590c341d9a041.png">

[image source](https://discuss.analyticsvidhya.com/t/how-decision-tree-performs-non-linear-classification/6001)

### Data

Let's work with [the adult census data set from kaggle](https://www.kaggle.com/uciml/adult-census-income). 

Predicting **income** as a binary classification problem (**above 50k** or **below 50k** income)

In [ ]:
adult_df_large = pd.read_csv("../data/adult.csv")
train_df, test_df = train_test_split(adult_df_large, test_size=0.6, random_state=42)

# Replace question marks in the dataset with NaN values
train_df_nan = train_df.replace("?", np.NaN)
test_df_nan = test_df.replace("?", np.NaN)
train_df_nan.head()

In [ ]:
numeric_features = ["age", "fnlwgt", "capital.gain", "capital.loss", "hours.per.week"]
categorical_features = [
    "workclass",
    "marital.status",
    "occupation",
    "relationship",
    "native.country",
]
ordinal_features = ["education"]
binary_features = ["sex"]
drop_features = ["race", "education.num"]
target_column = "income"

In [ ]:
education_levels = [
    "Preschool",
    "1st-4th",
    "5th-6th",
    "7th-8th",
    "9th",
    "10th",
    "11th",
    "12th",
    "HS-grad",
    "Prof-school",
    "Assoc-voc",
    "Assoc-acdm",
    "Some-college",
    "Bachelors",
    "Masters",
    "Doctorate",
]

In [ ]:
assert set(education_levels) == set(train_df["education"].unique())

In [ ]:
numeric_transformer = make_pipeline(StandardScaler())

ordinal_transformer = make_pipeline(
    OrdinalEncoder(categories=[education_levels], dtype=int)
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse=False),
)

binary_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(drop="if_binary", dtype=int),
)

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (ordinal_transformer, ordinal_features),
    (binary_transformer, binary_features),
    (categorical_transformer, categorical_features),
    ("drop", drop_features),
)

<br><br><br>
Create train and test sets.

In [ ]:
X_train = train_df_nan.drop(columns=[target_column])
y_train = train_df_nan[target_column]

X_test = test_df_nan.drop(columns=[target_column])
y_test = test_df_nan[target_column]

### Do we have class imbalance? 

- There is *a bit of* class imbalance.
- Let's use **accuracy** as our metric. 

In [ ]:
train_df_nan["income"].value_counts(normalize=True)

In [ ]:
scoring_metric = "accuracy"

Let's store all the results in a dictionary called `results`. 

In [ ]:
results = {}

<br><br><br><br><br>

-----------
[Study on your own]  
Main item to focus on: **what is stratified DummyClassifier?**


> “stratified”: the predict_proba method randomly samples one-hot vectors from a multinomial distribution parametrized by the empirical class prior probabilities. The predict method returns the class label which got probability one in the one-hot vector of predict_proba. Each sampled row of both methods is therefore independent and identically distributed.

Source: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

### Baselines

####  `DummyClassifier` baseline

In [ ]:
from utils import mean_std_cross_val_scores
dummy = DummyClassifier(strategy="stratified")
results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metric
)
results["Dummy"]

[End of study on your own]

---------
<br><br><br><br><br>

#### `DecisionTreeClassifier` baseline

- Let's try decision tree classifier on our data. 

In [ ]:
pipe_dt = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=123))
results["Decision tree"] = mean_std_cross_val_scores(
    pipe_dt, X_train, y_train, return_train_score=True, scoring=scoring_metric
)
pd.DataFrame(results).T

Decision tree is clearly **???**. 

<br><br><br><br>

## Random forests 

### General idea

- A single decision, without any constraints on the max-depth or number of leaves, tree is likely to overfit
- Use a collection of diverse decision trees
- Each tree *might overfit* on some part of the data but we can **reduce overfitting by AVERAGING or VOTING on the results** 

### `RandomForestClassifier` 

- Before understanding the details let's first try it out.  

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# ----- NEW CLASSIFIER: RandomForestClassifier -----
pipe_rf = make_pipeline(
    preprocessor, RandomForestClassifier(random_state=123, n_jobs=-1)
)
# --------------------------
results["Random forests"] = mean_std_cross_val_scores(
    pipe_rf, X_train, y_train, return_train_score=True, scoring=scoring_metric
)
pd.DataFrame(results).T

The validation scores are better although it seems likes we are still overfitting. 

### How do they work? 

- Decide **how many decision trees** we want to build
    - can control with `n_estimators` hyperparameter 
- `fit` a diverse set of that many decision trees by **injecting randomness** in the classifier construction
- `predict` by voting (classification) or averaging (regression) of predictions given by individual models

### Inject randomness in the classifier construction

To ensure that the trees in the random forest are different we inject randomness in two ways:  

1. **Data Bagging (Bootstrap Aggregation)**: Build each tree on a **random bootstrapped subset of samples** (i.e., a sample drawn **with replacement** from the training set)
2. **Feature Bagging**: At each node, select a **random subset of features** (controlled by `max_features` in `scikit-learn`) and look for the best possible test involving one of these features   

<br><br><br>

--------------

**Bootstrapping** is a statistical resampling technique that involves **random sampling** of a dataset **with replacement**. 
<br><br><br>

An example of a bootstrap samples
Suppose this is your original dataset: [1,2,3,4]
- a sample drawn with replacement: [1,1,3,4]
- a sample drawn with replacement: [3,2,2,2]
- a sample drawn with replacement: [1,2,4,4]
- ...

--------------

### The RandomForest classifier 

- Create a collection (ensemble) of trees. Grow each tree on an independent bootstrap sampling of the data.
- At each node:
    - At each node, **randomly and independently select a subset** of features out of all features
    - Find the best split on the selected features. 
    - Grow the trees to the specified `max_depth`.
- Prediction time    
    - Vote the trees to get predictions for new example. 

### Example 

- Let's create a random forest with **3 estimators** (3 trees). 
- I'm using `max_depth=2` for easy visualization. 

In [ ]:
pipe_rf_demo = make_pipeline(
    preprocessor, RandomForestClassifier(max_depth=2, n_estimators=3, random_state=123)
)
pipe_rf_demo.fit(X_train, y_train);

- Let's sample a test example. 

In [ ]:
test_example = X_test.sample(1)
print("Classes: ", pipe_rf_demo.classes_)
print("Prediction by random forest: ", pipe_rf_demo.predict(test_example))


--------------
[Study on your own]  
**Check how the transformer is transforming the features** 

Let's get the feature names of transformed features.

In [ ]:
feature_names = (
    numeric_features
    + ordinal_features
    + binary_features
    + list(
        pipe_rf_demo.named_steps["columntransformer"]
        .named_transformers_["pipeline-4"]
        .named_steps["onehotencoder"]
        .get_feature_names_out()
    )
)
feature_names[:10]

In [ ]:
transformed_example = preprocessor.transform(test_example)
pd.DataFrame(data=transformed_example.flatten(), index=feature_names)

[End of study on your own]

------------

- We can look at different trees created by random forest. 
- Note that each tree looks at different set of features and slightly different data. 

In [ ]:
from utils import display_tree

for i, tree in enumerate(
    pipe_rf_demo.named_steps["randomforestclassifier"].estimators_
):
    print("\n\nTree", i + 1)
    display(display_tree(feature_names, tree, counts=True))
    print("prediction", tree.predict(preprocessor.transform(test_example)))

### Some important hyperparameters:

- `n_estimators`: number of decision trees (higher = more complexity)
- `max_depth`: max depth of each decision tree (higher = more complexity)
- `max_features`: the number of features you get to look at each split (lower = more randomness = more diversity across estimators)


### Random forests: number of trees (`n_estimators`) and the fundamental tradeoff 

In [ ]:
from plotting_functions import make_num_tree_plot
make_num_tree_plot(
    preprocessor, X_train, y_train, X_test, y_test, [1, 5, 10, 25, 50, 100, 200, 500]
)

#### Number of trees and fundamental trade-off

- Above: seems like we're beating the fundamental "tradeoff" by increasing training score and not decreasing validation score much.
- This is the promise of ensembles, though it's not guaranteed to work so nicely.



**More trees are better! We pick less trees for speed.**

<br><br><br><br><br><br><br><br>

### Strengths and weaknesses of RandomForest

- Usually one of the best performing off-the-shelf classifiers without heavy tuning of hyperparameters
- Don't require scaling of data 
- Less likely to overfit 
- Slower than decision trees because we are fitting multiple trees but can easily **parallelize training** because all trees are independent of each other (that said, sklearn implementation is kind of slow)
- In general, able to capture a much broader picture of the data compared to a single decision tree. 

### Weaknesses

- Require more memory 
- Hard to interpret
- Tend not to perform well on high dimensional sparse data such as text data



<br><br><br>
**Warning**: It's a **Random**Forest, expect randomness! Make sure to set the `random_state` for reproducibility. Changing the `random_state` can have a big impact on the model and the results due to the random nature of these models. Having more trees can get you a more robust estimate. 


<img src=https://upload.wikimedia.org/wikipedia/en/4/4d/Leo_Breiman.jpg>

Leo Breiman, 1928 – 2005
- University of California, Berkeley
- classification and regression trees and ensembles of trees fit to bootstrap samples
- **Bootstrap aggregation** was given the name **Bagging** by Breiman
- **Random Forest**: [The original random forests paper](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)

<br><br>

## ❓❓ Questions for you

### (iClicker) Exercise 11.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) Every tree in a random forest uses a different bootstrap sample of the training set.
- (B) To train a tree in a random forest, we first randomly select a subset of features. The tree is then restricted to only using those features.
- (C) A reasonable implementation of `predict_proba` for random forests would be for each tree to "vote" and then normalize these vote counts into probabilities.
- (D) Decreasing the hyperparameter max_features (the number of features to consider for a split) doesn’t allow each estimator (tree) to fit too closely to the data.
- (E) A random forest with only one tree is likely to get a higher training error than a decision tree of the same depth.

--------------

**[Bonus]**



[`ExtraTreesClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)
adds more randomness choosing random splits for each feeature.

Difference between RandomForest and ExtraTrees:
- RandomForest chooses the optimum split while ExtraTrees chooses it randomly. 
- However, once the split points are selected, the two algorithms choose the best one between all the subset of features. So, Extra Trees adds randomization during splitting but still does optimization during feature selection.


[End of bonus]

---------------

<br><br><br><br>

## Gradient boosted trees 

Another popular and effective class of tree-based models is gradient boosted trees. 

- No randomization.
- The key idea is combining many simple models called weak learners to create a strong learner. 
- They combine multiple shallow (depth 1 to 5) decision trees  
- They build trees in a **serial manner** (NOT independent in parallel), where each tree tries to **correct the mistakes** of the previous one. 

### Important hyperparameters

- `n_estimators`
    - control the number of trees to build
- `learning_rate`
    - controls how strongly each tree tries to correct the mistakes of the previous trees
    - **lower** learning rate 
      - **slows down** the learning in the gradient boosting model by making less corrections for each tree added to the model. 
      - This in turn results in **more trees** that must be added to the model.

We'll not go into the details. We'll look at brief examples of using the following three gradient boosted tree models. 

- [XGBoost](https://xgboost.readthedocs.io/en/latest/)
- [LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
- [CatBoost](https://catboost.ai/docs/concepts/python-quickstart.html)

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*PJXOO3x2HC_XfnxvjN-dEg.png">

Source of the above image, and a good overview blog post on the 3 models: [check here](https://towardsdatascience.com/catboost-vs-lightgbm-vs-xgboost-c80f40662924#:~:text=In%20CatBoost%2C%20symmetric%20trees%2C%20or,the%20same%20depth%20can%20differ.)

### [XGBoost](https://xgboost.ai/about) 

- Not part of `sklearn` but has similar interface. 
- Install it in your conda environment: `conda install -c conda-forge xgboost`
- Supports missing values
- GPU training, networked parallel training
- Supports sparse data
- Typically better scores than random forests    

### [LightGBM](https://lightgbm.readthedocs.io/)

- Its name (Light) implies a lighter model and more speed, and yes, it is **faster**
- Not part of `sklearn` but has similar interface. 
- Install it in your conda environment: `conda install -c conda-forge lightgbm`
- Small model size
- Faster 
- Typically better scores than random forests

### [CatBoost](https://catboost.ai/)

> CatBoost is an open-source machine learning(gradient boosting) algorithm, with its name coined from “Category” and “Boosting.” It was developed by Yandex (Russian Google ) in 2017. According to Yandex, CatBoost has been applied to a wide range of areas such as recommendation systems, search ranking, self-driving cars, forecasting, and virtual assistants. It is the successor of MatrixNet that was widely used within Yandex products.  
[https://neptune.ai/blog/when-to-choose-catboost-over-xgboost-or-lightgbm]

- Name comes from its focus on **Categorical** features
- Not part of `sklearn` but has similar interface. 
- Install it in your conda environment: `conda install -c conda-forge catboost`
- CatBoost supports distributed training across GPUs
- Usually better scores but slower compared to `XGBoost` and `LightGBM`     

In [ ]:
!conda install -c conda-forge xgboost -y
!conda install -c conda-forge lightgbm -y
!conda install -c conda-forge catboost -y

In [ ]:
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

pipe_lr = make_pipeline(
    preprocessor, LogisticRegression(max_iter=2000, random_state=123)
)
pipe_dt = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=123))
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=123))
pipe_xgb = make_pipeline(
    preprocessor, XGBClassifier(random_state=123, eval_metric="logloss", verbosity=0)
)
pipe_lgbm = make_pipeline(preprocessor, LGBMClassifier(random_state=123))
pipe_catboost = make_pipeline(
    preprocessor, CatBoostClassifier(verbose=0, random_state=123)
)
classifiers = {
    "logistic regression": pipe_lr,
    "decision tree": pipe_dt,
    "random forest": pipe_rf,
    # "XGBoost": pipe_xgb,
    "LightGBM": pipe_lgbm,
    "CatBoost": pipe_catboost,
}

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=DeprecationWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
results = {}

In [ ]:
adult_df_large = pd.read_csv("../data/adult.csv")
train_df, test_df = train_test_split(adult_df_large, test_size=0.6, random_state=42)

# Replace question marks in the dataset with NaN values
train_df_nan = train_df.replace("?", np.NaN).dropna()
test_df_nan = test_df.replace("?", np.NaN).dropna()
train_df_nan.head()

X_train = train_df_nan.drop(columns=[target_column])
y_train = train_df_nan[target_column]

X_test = test_df_nan.drop(columns=[target_column])
y_test = test_df_nan[target_column]

In [ ]:
y_train_xgboost = y_train.copy()
y_train_xgboost[y_train_xgboost == '<=50K'] = 0
y_train_xgboost[y_train_xgboost == '>50K'] = 1

In [ ]:
y_train_xgboost

In [ ]:
dummy = DummyClassifier(strategy="stratified")
results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metric
)

In [ ]:
for (name, model) in classifiers.items():
    print(name)
    if name == 'XGBoost':
        results[name] = mean_std_cross_val_scores(
            model, X_train, y_train_xgboost, return_train_score=True, scoring=scoring_metric
        )
    else:
        results[name] = mean_std_cross_val_scores(
            model, X_train, y_train, return_train_score=True, scoring=scoring_metric
        )

In [ ]:
pd.DataFrame(results).T

**Some observations**
- Keep in mind that all these results are with default hyperparameters
- Ideally we would carry out hyperparameter optimization for all of them and then compare the results. 
- We are using a particular scoring metric (accuracy in this case)
- We are scaling numeric features but it shouldn't matter for these tree-based models. 
- Look at the std. Doesn't look very high. 
    - The scores look more or less stable. 

In [ ]:
pd.DataFrame(results).T

- Decision trees and random forests overfit
    - Other models do not seem to overfit much. 
- Fit times
    - Decision trees are fast but not very accurate
    - LightGBM is faster than decision trees and more accurate! 
    - CatBoost fit time is highest followed by random forests.  
    - There is not much difference between the validation scores of XGBoost, LightGBM, and CatBoost but it is about 48x slower than LightGBM!
    - XGBoost and LightGBM are faster and more accurate than random forest!    
- Scores times  
    - Prediction times are much smaller in all cases. 

### What classifier should I use?

**Simple answer**
- Whichever gets the highest CV score making sure that you're not overusing the validation set.

<br><br><br><br>

## Averaging 

Earlier we looked at a bunch of classifiers:

In [ ]:
classifiers.keys()

What if we use all these models and let them vote during prediction time? 

In [ ]:
from sklearn.ensemble import VotingClassifier

averaging_model = VotingClassifier(
    list(classifiers.items()), voting="soft"
)  # need the list() here for cross_val to work!

In [ ]:
from sklearn import set_config

set_config(display="diagram")  # global setting

In [ ]:
averaging_model

This `VotingClassifier` will take a _vote_ using the predictions of the constituent classifier pipelines.

Main parameter: `voting`
- `voting='hard'` 
    - it uses the output of `predict` and actually votes.
- `voting='soft'`
    - with `voting='soft'` it averages the output of `predict_proba` and then thresholds / takes the larger.

- The choice depends on whether you trust `predict_proba` from your base classifiers - if so, it's nice to access that information.

In [ ]:
averaging_model.fit(X_train, y_train);

- What happens when you `fit` a `VotingClassifier`?
    - It will fit all constituent models.    

```{note}
It seems sklearn requires us to actually call `fit` on the `VotingClassifier`, instead of passing in pre-fit models. This is an implementation choice rather than a conceptual limitation.
```

Let's look at particular test examples where `income` is ">50k" (y=1):

In [ ]:
test_g50k = (
    test_df.query("income == '>50K'").sample(4, random_state=2).drop(columns=["income"])
)
test_l50k = (
    test_df.query("income == '<=50K'")
    .sample(4, random_state=2)
    .drop(columns=["income"])
)

In [ ]:
averaging_model.classes_

In [ ]:
data = {"Voting classifier": averaging_model.predict(test_l50k)}
pd.DataFrame(data)

For hard voting, these are the votes:

In [ ]:
r1 = {
    name: classifier.predict(test_l50k)
    for name, classifier in averaging_model.named_estimators_.items()
}
data.update(r1)
pd.DataFrame(data)

For soft voting, these are the scores:

In [ ]:
r1 = {
    name: classifier.predict_proba(test_l50k)
    for name, classifier in averaging_model.named_estimators_.items()
}
r1

Let's see how well this model performs.

In [ ]:
results["Voting"] = mean_std_cross_val_scores(averaging_model, X_train, y_train)

In [ ]:
pd.DataFrame(results).T

It appears that here we didn't do much better than our best classifier :(. 

Let's try removing decision tree classifier. 

In [ ]:
classifiers_ndt = classifiers.copy()
del classifiers_ndt["decision tree"]
averaging_model_ndt = VotingClassifier(
    list(classifiers_ndt.items()), voting="soft"
)  # need the list() here for cross_val to work!

results["Voting_ndt"] = mean_std_cross_val_scores(
    averaging_model_ndt,
    X_train,
    y_train,
    return_train_score=True,
    scoring=scoring_metric,
)

In [ ]:
pd.DataFrame(results).T

Still the results are not better than the best performing model. 

- It didn't happen here but how could the average do better than the best model???
  - From the perspective of the best estimator (in this case CatBoost), why are you adding on worse estimators??


Here's how this can work:

| Example | log reg    | rand forest    | cat boost    | Averaged model |
|--------|--------|--------|---------|---------------|
|  1     | ✅    |   ✅    | ❌     | ✅✅❌=>✅  |
|  2     | ✅    |   ❌    | ✅     | ✅❌✅=>✅  |
|  3     | ❌    |   ✅    | ✅     | ❌✅✅=>✅  |


**In short, as long as the different models make different mistakes, this can work.**

Why not always do this?

1. `fit`/`predict` time.
2. Reduction in interpretability.
3. Reduction in code maintainability (e.g. Netflix prize).

### What kind of estimators can we combine? 

- You can combine 
    - completely different estimators, or similar estimators.
    - estimators trained on different samples.
    - estimators with different hyperparameter values.

<br><br><br><br>

## Stacking (Meta-learning model)


- Another type of ensemble is stacking.
- Instead of averaging the outputs of each estimator, instead use their outputs as _inputs to another model_.
- In sklearn, the `StackingClassifier` uses logistic regression for classification as the `final_estimator`
  - We don't need a complex model here necessarily, more of a weighted average.
  - The features going into the logistic regression are the classifier outputs, _not_ the original features!
  - So the number of coefficients = the number of base estimators!

In [ ]:
from sklearn.ensemble import StackingClassifier

The code starts to get too slow here; so we'll remove CatBoost. 

In [ ]:
classifiers_nocat = classifiers.copy()
del classifiers_nocat["CatBoost"]

In [ ]:
stacking_model = StackingClassifier(list(classifiers_nocat.items()))

In [ ]:
stacking_model.fit(X_train, y_train);

What's going on in here? 

- It is doing cross-validation by itself by default (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html))
  - It is fitting the base estimators on the training fold
  - And the predicting on the validation fold
  - And then fitting the meta-estimator on that output (on the validation fold)

 > Note that estimators_ are fitted on the full X while final_estimator_ is trained using cross-validated predictions of the base estimators using cross_val_predict.

Here is the input features (X) to the meta-model:

In [ ]:
valid_sample = train_df.sample(4, random_state=2).drop(columns=["income"])

In [ ]:
r3 = {
    name: pipe.predict_proba(valid_sample)
    for (name, pipe) in stacking_model.named_estimators_.items()
}
r3

- Our meta-model is logistic regression (which it is by default).
- Let's look at the learned coefficients. 

In [ ]:
pd.DataFrame(
    data=stacking_model.final_estimator_.coef_[0],
    index=classifiers_nocat.keys(),
    columns=["Coefficient"],
)

In [ ]:
stacking_model.final_estimator_.intercept_

- It seems that the LightGBM is being trusted the most. 

In [ ]:
stacking_model.predict(test_g50k)

In [ ]:
stacking_model.predict_proba(test_g50k)

(This is the `predict_proba` from logistic regression)

Let's see how well this model performs.

In [ ]:
results["Stacking_nocat"] = mean_std_cross_val_scores(
    stacking_model, X_train, y_train, return_train_score=True, scoring=scoring_metric
)

In [ ]:
pd.DataFrame(results).T

- The situation here is a bit mind-boggling.
- On each fold of cross-validation it is doing cross-validation.
- This is really loops within loops within loops within loops...

- We can also try a different final estimator:
- Let's `DecisionTreeClassifier` as a final estimator. 

In [ ]:
stacking_model_tree = StackingClassifier(
    list(classifiers_nocat.items()), final_estimator=DecisionTreeClassifier(max_depth=3)
)

The results are not very good. But we can look at the tree:

In [ ]:
stacking_model_tree.fit(X_train, y_train);

In [ ]:
display_tree(list(classifiers_nocat.keys()), stacking_model_tree.final_estimator_)

#### An effective strategy

- Randomly generate a bunch of models with different hyperparameter configurations, and then stack all the models.

- What is an advantage of ensembling multiple models as opposed to just choosing one of them?
    - You may get a better score.
- What is an disadvantage of ensembling multiple models as opposed to just choosing one of them?
    - Slower, more code maintenance issues.

------------
<br><br><br><br><br><br>
## Final thoughts on Ensemble models

<img src="https://vitalflux.com/wp-content/uploads/2022/11/boosting-vs-bagging-differences-examples-640x332.png">



- **Bagging**: Often considers homogeneous weak learners, learns them independently from each other in parallel and combines them following some kind of averaging process
  - Bagging reduces Variance (bagging technique tries to resolve the issue of overfitting training data)
- **Boosting**: Often considers homogeneous weak learners, learns them **sequentially** in a very adaptative way (a base model depends on the previous ones) and combines them
  - Boosting reduces Bias
- **Stacking**: Often considers heterogeneous weak learners, learns them in parallel and combines them by training a meta-model (final estimator) to output a prediction based on the different weak models predictions


<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/0*e-na5r7mF8lVAfPK.png">
-------------

<br><br><br><br><br><br>

### Question 11.3

Is this bagging or boosting?

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/0*KcQZgCA34Cggnrso.png">

## Question 11.4

Is this bagging or boosting?

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*alEu0ArH61Hfz2k2wQHNnA.png">

## Summary 

- You have a number of models in your toolbox now.  
- Ensembles are usually pretty effective.
  - Tree-based classifiers are particularly popular and effective on a wide range of problems. 
  - But they trade off code complexity and speed for prediction accuracy.
  - Don't forget that hyperparameter optimization multiplies the slowness of the code!
- Stacking is a bit slower than voting.
  - As a bonus, you get to see the coefficients for each base classifier.
- Most of the above approaches have regression models as well as classification models.

#### Relevant papers

- [Fernandez-Delgado et al. 2014](http://jmlr.org/papers/volume15/delgado14a/delgado14a.pdf) compared 179 classifiers on 121 datasets:
    - First best class of methods was Random Forest and second best class of methods was (RBF) SVMs.